In [66]:
import pandas as pd
import numpy as np
import scipy.io
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow import keras
from tensorflow.keras import layers
from datasetResize import *
from sklearn.metrics import classification_report


In [48]:
# lecture du excel
labels_df = pd.read_csv('REFERENCE-V3.csv', header=None)
labels_df.columns = ['filename', 'label']

print(labels_df.head())

  filename label
0   A00001     N
1   A00002     N
2   A00003     N
3   A00004     A
4   A00005     A


In [50]:
# Charger tous les signaux et leurs labels
labels = []
signals = []
signals_padded = []
signals_truncated = []
signals_interpolated = []

# Initialiser avec les paramètres pour le redimensionnement
ecg_resizer_max = ECGResizing(target_length=18286)
ecg_resizer_med = ECGResizing(target_length=9000)

for index, row in labels_df.iterrows():
    
    labels.append(row['label'])
    filename = row['filename']
    # Charger un signal ECG
    signal = ecg_resizer_max.load_ecg(f'training2017/{filename}.mat')
    #signals = np.append(signals,[[signal]])
    signals.append(signal)

    # Appliquer le zero padding
    signal_padded = ecg_resizer_max.resize_signal(signal, method='padding')
    signals_padded.append(signal_padded)

    # Appliquer le tronquage
    signal_truncated = ecg_resizer_med.resize_signal(signal, method='padding')
    signals_truncated.append(signal_truncated)

    # Appliquer l'interpolation
    signal_interpolated = ecg_resizer_med.resize_signal(signal, method='interpolate')
    signals_interpolated.append(signal_interpolated)



In [51]:
print(len(signals[0]))
print(len(signals_padded[0]))
print(len(signals_truncated[0]))
print(len(signals_interpolated[0]))

9000
18286
9000
9000


In [52]:
labels = np.array(labels)
signals_padded = np.array(signals_padded)
signals_truncated = np.array(signals_truncated)
signals_interpolated = np.array(signals_interpolated)


# Encoder les labels
label_mapping = {'N': 0, 'A': 1, 'O': 2, '~': 3}
labels_encoded = np.vectorize(label_mapping.get)(labels)


In [62]:
# Diviser en ensembles d'entraînement et de test
#X_train_signals, X_test_signals, y_train_signals, y_test_signals = train_test_split(signals, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_padded, X_test_padded, y_train_padded, y_test_padded = train_test_split(signals_padded, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_truncated, X_test_truncated, y_train_truncated, y_test_truncated = train_test_split(signals_truncated, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_interpolated, X_test_interpolated, y_train_interpolated, y_test_interpolated = train_test_split(signals_interpolated, labels_encoded, test_size=0.2, stratify=labels, random_state=42)

# Afficher les formes des ensembles
print(f'Ensemble d\'entraînement : {X_train_padded.shape}, Ensemble de test : {X_test_padded.shape}')

Ensemble d'entraînement : (6822, 18286), Ensemble de test : (1706, 18286)


In [80]:
# Créer le modèle MLP
model_padded = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_padded.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_padded.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model_padded.summary()

c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 128)            │     2,340,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,344,996 (8.95 MB)

 Trainable params: 2,344,996 (8.95 MB)

 Non-trainable params: 0 (0.00 B)

In [81]:
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_padded), y=y_train_padded)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history = model_padded.fit(X_train_padded, y_train_padded, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)


Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.2607 - loss: 436.9254 - val_accuracy: 0.2945 - val_loss: 187.3676
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3275 - loss: 85.1002 - val_accuracy: 0.1604 - val_loss: 80.9074
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2849 - loss: 31.6229 - val_accuracy: 0.1612 - val_loss: 64.1515
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4716 - loss: 14.1036 - val_accuracy: 0.4418 - val_loss: 51.4643
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5790 - loss: 13.5327 - val_accuracy: 0.4659 - val_loss: 43.2177
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5980 - loss: 4.3711 - val_accuracy: 0.5055 - val_loss: 33.2677
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6247 - loss: 2.2423 - val_accuracy: 0.5055 - val_loss: 30.1341
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6268 - loss: 1.7914 - v

In [82]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_padded.evaluate(X_test_padded, y_test_padded)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_padded.predict(X_test_padded)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_padded, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5339 - loss: 23.3233
Précision sur l'ensemble de test : 0.5281
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.84      0.70      1015
           A       0.11      0.04      0.06       152
           O       0.26      0.06      0.10       483
           ~       0.08      0.16      0.10        56

    accuracy                           0.53      1706
   macro avg       0.26      0.28      0.24      1706
weighted avg       0.45      0.53      0.45      1706



In [83]:
# Créer le modèle MLP
model_truncated = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_truncated.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model_truncated.summary()

c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 128)            │     1,152,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,156,388 (4.41 MB)

 Trainable params: 1,156,388 (4.41 MB)

 Non-trainable params: 0 (0.00 B)

In [84]:
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_truncated.fit(X_train_truncated, y_train_truncated, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2838 - loss: 448.0167 - val_accuracy: 0.1817 - val_loss: 205.6044
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2666 - loss: 93.2023 - val_accuracy: 0.1971 - val_loss: 53.4403
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3151 - loss: 15.3863 - val_accuracy: 0.2762 - val_loss: 21.8410
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3740 - loss: 3.3583 - val_accuracy: 0.2264 - val_loss: 16.2983
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3351 - loss: 3.5277 - val_accuracy: 0.3033 - val_loss: 11.5519
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3646 - loss: 2.0782 - val_accuracy: 0.2711 - val_loss: 10.9956
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3588 - loss: 1.4664 - val_accuracy: 0.2520 - val_loss: 11.6455
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5228 - loss: 1.4309 - val_

In [85]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_truncated.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_truncated.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5435 - loss: 4.9660
Précision sur l'ensemble de test : 0.5422
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.90      0.72      1015
           A       0.08      0.01      0.02       152
           O       0.24      0.01      0.02       483
           ~       0.04      0.09      0.05        56

    accuracy                           0.54      1706
   macro avg       0.24      0.25      0.20      1706
weighted avg       0.43      0.54      0.44      1706



In [86]:
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_interpolated), y=y_train_interpolated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_interpolated = model_truncated.fit(X_train_interpolated, y_train_interpolated, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3800 - loss: 15.4614 - val_accuracy: 0.2821 - val_loss: 3.5462
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2851 - loss: 6.4191 - val_accuracy: 0.2894 - val_loss: 2.0263
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2966 - loss: 1.3796 - val_accuracy: 0.2879 - val_loss: 2.0317
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2635 - loss: 1.3591 - val_accuracy: 0.0850 - val_loss: 2.0431
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3488 - loss: 1.3387 - val_accuracy: 0.0850 - val_loss: 2.0517
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4720 - loss: 1.3880 - val_accuracy: 0.5905 - val_loss: 2.0580
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3704 - loss: 1.3936 - val_accuracy: 0.5905 - val_loss: 2.0625
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6002 - loss: 1.3574 - val_accuracy: 0

In [87]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_truncated.evaluate(X_test_interpolated, y_test_interpolated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_truncated.predict(X_test_interpolated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_interpolated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5824 - loss: 2.0256
Précision sur l'ensemble de test : 0.5856
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.98      0.74      1015
           A       0.00      0.00      0.00       152
           O       0.12      0.00      0.01       483
           ~       0.00      0.00      0.00        56

    accuracy                           0.59      1706
   macro avg       0.18      0.25      0.19      1706
weighted avg       0.39      0.59      0.44      1706

